# Download audio file


In [222]:
import youtube_dl

import re
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

import subprocess

import scipy.io.wavfile as wav
import librosa

from mega import Mega
import getpass

import warnings; warnings.simplefilter('ignore')

## Create a pandas dataset

In [271]:
df=pd.read_csv(os.path.abspath(os.path.join(os.getcwd(),os.pardir,os.pardir))+'/data/data_split.csv',index_col=0)

In [216]:
df.sample(10)

,url,start_time,end_time,class_label
5569,eLmWZL2_r80,30.0,40.0,Humans
11420,lK2fHm0V04o,410.0,420.0,Natural
9246,XsZ9ndrYPxI,0.0,10.0,Humans
6712,ehmd1gJM890,10.0,20.0,Humans
6348,ZUgJXeToltg,200.0,210.0,Humans
10565,1zoYJVhzDGc,30.0,40.0,Natural
336,R3C1n611idY,90.0,100.0,Animal
1278,QkpNIF8xzEE,370.0,380.0,Animal
10623,01pUDNKXK9c,220.0,230.0,Natural
6706,lcwMqeA7fT0,30.0,40.0,Humans


In [3]:
df['class_label'].value_counts()

Animal     5000
Natural    5000
Humans     5000
Name: class_label, dtype: int64

## Download Audio => Extract Features => Upload on Mega


In [118]:
def get_audio_features(filename:str)-> np.array:
    '''
    input: filename (.vaw file)
    output: dictionary containing mffc and mel-spectogram features a
    '''
    hop_length = 512
    
    y, sr = librosa.load(filename)
    
    mfcc_ = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13,n_fft=513)
    mel_spect = librosa.feature.melspectrogram(y=y,sr=sr,n_fft=513,win_length=400)
    
    return dict(mfcc = mfcc_, mel_spec = mel_spect)

In [272]:
def save_on_mega(file:str ,m: Mega):
    '''
    save data in folder 'features_'
    the folder has been manually created on Mega website
    '''
    folder = m.find('features_')
    m.upload(file, folder[0])

In [185]:
#login to Mega account 

m = Mega()
email = input('insert email ')
psw = getpass.getpass(prompt='insert password')
m.login(email,psw)

insert email andrea.faviait@gmail.com
insert password········


In [ ]:
not_downloaded = dict(Animal = 0, Humans = 0, Natural = 0)

#n.b. you might have to chance the working directory (os.chdir()))

for i, row in tqdm(df.iterrows()):
    
    url = 'https://www.youtube.com/watch?v=' + row['url']
    file_name = str(i)+"_"+row['class_label']
    
    try:
        
        #download youtube video & create a clipped .wav file
        subprocess.Popen("ffmpeg -ss " + str(row['start_time']) + " -i $(youtube-dl -f 140 --get-url " + url + ") -t 10 -c:v copy -c:a copy " + file_name + ".mp4", shell=True).wait()
        subprocess.Popen("ffmpeg -i "+file_name+".mp4 -ab 160k -ac 2 -ar 44100 -vn "+file_name+'.wav',shell=True).wait()
        
        #extract mfcc, mel features
        res = get_audio_features(file_name+'.wav')
        
        #save .npy file and upload on mega
        file = np.save(file_name,res)
        save_on_mega(file_name+'.npy',m)
        
        #remove .mp4, .wav files, .npy file
        subprocess.Popen('rm '+file_name+'.mp4',shell=True).wait()
        subprocess.Popen('rm '+file_name+'.wav',shell=True).wait()
        subprocess.Popen('rm '+file_name+'.npy',shell=True).wait()


    except Exception as e:
        not_downloaded[row['class_label']] += 1
        pass
    